<a href="https://colab.research.google.com/github/m5l14i11/walkthrough/blob/master/Copy_of_dynamic_programming_knapsack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knapsack Problem

__Inputs:__ Weight limit $W$, list of item weights $[w_1, \ldots, w_k]$, and list of item values $[v_1, \ldots, v_k]$.

__Output:__ Choose how many of each item to take $[n_1, \ldots, n_k]$ so that 
   1. Total weight is under the knapsack weight limit: $n_1 w_1 + \cdots + n_k w_k \leq W$.
   2. The value of stolen goods is maximized: $n_1 v_1 + \ldots + n_k v_k $ is max.

In [ ]:
W = 200
weights = [1, 5, 20, 35, 90]
values = [15, 14.5, 19.2, 19.8, 195.2]

## 1. Identify the optimal substructure

Suppose the current weight limit is $W$. Let us commit to stealing one of the available items and look at what is left to do.

   1. Suppose we commit to stealing item $j$.
   2. We now need to solve the same problem but for weight limit $W - w_j$. If the solution for this subproblem is obtained, then the original problem's solution is to take the solution for $W-w_j$ and append item $j$ to it.
   
We can thus see that the problem has optimal substructure.

## 2. Recurrence

$$\text{maxSteal}(W) = \max\ \left\{ \begin{array}{ll}
0 & \leftarrow\ \text{Choose to steal nothing and Quit!}\\
v_1 + \text{maxSteal}(W - w_1) & \leftarrow\ \text{Choose one unit of item}\ 1 \\
v_2 + \text{maxSteal}(W - w_2) & \leftarrow\ \text{Choose one unit of item}\ 2 \\
\vdots & \\
v_k + \text{maxSteal}(W - w_k) & \leftarrow\ \text{Choose one unit of item}\ k\\
\end{array} \right.$$

Base Case:

  * $\text{maxSteal}(0) = 0$ 
  * $\text{maxSteal}(W) = -\infty$ if $W < 0$.
 
 


In [ ]:
def maxSteal(W, weights, values):
    if W == 0: 
        return 0
    if W < 0:
        return 100000
    k = len(weights)
    assert len(values) == k
    opts = [ values[i] + maxSteal(W - weights[i], weights, values) for i in range(k) ]
    return max(opts)

In [ ]:
#WARNING: This will run for a very very long time.
print(maxSteal(25, weights, values))
print(maxSteal(W, weights, values))

100555.2


KeyboardInterrupt: ignored

## 3. Memoize

Memoization is very simple. We make a table $T[0], ... , T[W]$ for storing $\text{maxSteal}(j)$ for j ranging from $0$ to $W$.
The rest just follows the structure of the recurrence taking care to handle -ve values for weight separately.

## 4. Recover Solution

We store in a separate table $S[0], \ldots, S[W]$ which option provides us with the best value.


In [ ]:
def maxSteal_memo(W, weights, values):
    # Initialize the tables
    T = [0]* (W+1)
    S = [-1]* (W+1)
    k = len(weights)
    assert len(values) == k
    for w in range(1, W+1):
        opts =  [  ( (values[i]+ T[ w - weights[i] ]), i )  for i in range(k) if w - weights[i] >= 0 ]
        opts.append( (1000000, -1) ) # In case opts was empty from the previous step.
        T[w], S[w] = min(opts)
    # This finishes the computation
    stolen_item_ids = []
    weight_remaining = W
    while weight_remaining >= 0:
        item_id = S[weight_remaining]
        if item_id >= 0:
            stolen_item_ids.append('Steal Item ID %d: weight = %d, value = %f' % (item_id, weights[item_id], values[item_id]) )
            weight_remaining = weight_remaining - weights[item_id]
        else:
            break
    return T[W], stolen_item_ids

In [ ]:
print(maxSteal_memo(25, weights, values))
print(maxSteal_memo(W, weights, values))

(33.7, ['Steal Item ID 1: weight = 5, value = 14.500000', 'Steal Item ID 2: weight = 20, value = 19.200000'])
(132.7, ['Steal Item ID 1: weight = 5, value = 14.500000', 'Steal Item ID 3: weight = 35, value = 19.800000', 'Steal Item ID 3: weight = 35, value = 19.800000', 'Steal Item ID 3: weight = 35, value = 19.800000', 'Steal Item ID 2: weight = 20, value = 19.200000', 'Steal Item ID 3: weight = 35, value = 19.800000', 'Steal Item ID 3: weight = 35, value = 19.800000'])
